In [1]:
from heapq import heappush, heappop
import pandas as pd
import re
import folium
import geohash
import random
import requests

In [2]:
# Required Files, those are not public files
# geohashmap = pd.read_csv('*') # 2843 rows × 1714 columns
# speedmap = pd.read_csv('*')
# ttime_lambda = pd.read_csv('*')

In [3]:
ttime_lambda = ttime_lambda.to_dict()
time_lambda = {}
for i in range(len(ttime_lambda['column'])):
    minuate = int(ttime_lambda['column'][i])
    datae = re.findall(r"'(.*?)'", ttime_lambda['tuple'][i])
    if minuate not in time_lambda:
        time_lambda[minuate] = [datae]
    else: time_lambda[minuate].append(datae)
        
# decoding function
base = {}
for i in range(32):
    if i < 10:
        base[str(i)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]
    elif i < 17:
        base[chr(i+88)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]
    elif i < 19:
        base[chr(i+89)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]
    elif i < 21:
        base[chr(i+90)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]
    else:
        base[chr(i+91)] = '0' * (5-len(bin(i)[2:])) + bin(i)[2:]

def hashtoarrayindex(hash):
    bi = ''
    for c in hash:
        bi += base[c]
    odd = '0b' # row
    even = '0b' # column
    for i in range(len(bi)//2):
        odd += bi[2*i+1]
        even += bi[2*i]
    return (int(odd, 2)-733067, int(even, 2)-900017)

NameError: name 'ttime_lambda' is not defined

In [170]:
# entity
class Customer(object):
    def __init__(self, index, time, up_pos, down_pos):
        self.index = index
        self.time = time
        self.up_pos = up_pos
        self.down_pos = down_pos

class Car(object):
    def __init__(self, index, capacity, reserved, now_pos, path, passed_capacity):
        self.index = index
        self.capacity = capacity # [customer 객체]
        self.reserved = reserved # [customer 객체]
        self.now_pos = now_pos
        self.path = path # [(목적지, 승객을 태울건지)]
        self.passed_capacity = passed_capacity
        
car_max_cap = 20

In [171]:
def cal_path(start, end):
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    headers = {
        # "Authorization": "kakao api key"
    }
    params = {
        "origin": f"{start[1]},{start[0]}",
        "destination": f"{end[1]},{end[0]}",
        "waypoints": "",
        "road_details": "True"
    }

    response = requests.get(url, headers=headers, params=params)
    response = response.json()
    print("cal_path", response['routes'][0]['result_msg'])
    
    all_vertices = []
    for route in response['routes']:
        for section in route['sections']:
            for road in section['roads']:
                all_vertices.extend(road['vertexes'])
    vertices_tuples = [(all_vertices[i+1], all_vertices[i]) for i in range(0, len(all_vertices), 2)][::2]
    return vertices_tuples

def cal_speed_path(start_pos, end_pos): # 시작 지점과 끝 지점을 제외한 경로를 만들어주는 함수
    path = []
    freq = 20
    sx, sy = start_pos
    ex, ey = end_pos
    flipx, flipy = 1, 1
    if abs(ey-sy) >= abs(ex-sx):
        period = abs(ey-sy)//freq
        if ex < sx: flipx = -1
        if ey < sy: flipy = -1
        for i in range(period):
            for j in range(abs(sx-ex)):
                temp = (sx + j*flipx, sy + (i+1)*freq*flipy)
                temp_hash = geohashmap.iloc[temp]
                if temp_hash != str(0):
                    path.append(temp)
                    break
    else:
        period = abs(ex-sx)//freq
        if ex < sx: flipx = -1
        if ey < sy: flipy = -1
        for i in range(period):
            for j in range(abs(sy-ey)):
                temp = (sx + (i+1)*freq*flipx, sy + j * flipy)
                temp_hash = geohashmap.iloc[temp]
                if temp_hash != str(0):
                    path.append(temp)
                    break
    return path

def cal_time(start, end):
    dx = abs(start[0] - end[0]) 
    dy = abs(start[1] - end[1])
    # 30.944622355745157 dy 19.047461054233576 dx
    distance = dx * 19 + dy * 31 # 포항시 2843 rows × 1714 columns 기준
    vx, vy = speedmap.iloc[start], speedmap.iloc[end] # km/h
    if vx == 0: vx = 48.95 # 혹여나 데이터가 없는 경우 전체 평균 속력으로 사용
    if vy == 0: vy = 48.95
    vmean = (vx + vy)/2
    t = distance / (vmean * 1000/3600) # m/s로 바꿈
    if t <= 60:
        return 60
    return int((t//60) * 60)

def cal_total_time(start, end):
    start = hashtoarrayindex(start)
    end = hashtoarrayindex(end)
    temp_path = cal_speed_path(start,end)
    temp_path = [start] + temp_path + [end]
    total_time = 0
    for i in range(len(temp_path)-1):
        total_time += cal_time(temp_path[i], temp_path[i+1])
    return total_time

In [172]:
area_inf = [(-1, 1150), (1150, 1200), (1000, 1300), (1180, 1300)] # 이 값보다 작으면
area_dir = [(-1, -1), (1, 1), (-1, 1), (-1, 1)] # 1

def cal_area_num(dis_tuple):
    area = -1
    x, y = dis_tuple
    for areai, areainf in enumerate(area_inf):
        signx, signy = area_dir[areai][0], area_dir[areai][1]
        if areainf[0] * signx > x * signx and areainf[1] * signy > y * signy:
            area = areai
    return area

In [230]:
def cal_distance(hash1, hash2):
    x1, y1 = hashtoarrayindex(hash1)
    x2, y2 = hashtoarrayindex(hash2)
    return abs(x1-x2) * 19 + abs(y1-y2) * 31

def algorithm(customers, cars): # path를 직접 바꾸지 않도록
    des = [] # 차량의 목적지를 저장, 차량 당 한 곳만 지정
    for i in range(car_number): # 차량들의 목적지를 저장
        car = cars[i]
        now_pos = car.now_pos
        # now_area = cal_area_num(hashtoarrayindex(now_pos))
        now_area = i # i번 차량이 i 번 구역에 있다고 가정
        candidates = []
        if len(car.passed_capacity) > 0 and max(car.passed_capacity) > 6:
            max_index = car.passed_capacity.index(max(car.passed_capacity))
            des.append(car.capacity[max_index].down_pos)
            print(i,"th car", "go to let down to late boarding customer")
            continue
        if len(car.capacity) + len(car.reserved) < car_max_cap: 
            # 새로운 승객은 차에 공간이 있을 때만 태운다, 또는 예약된 승객이나 탑승한 승객이 너무 오래 기다린 경우 제외해야함
            for i, row in enumerate(customers):
                for j, element in enumerate(row):
                    if element[1] == now_area and element[2] in [now_area, now_area + 1]: # now_area가 시작점이면서 now_area or now_area + 1가 도착점인 승객의 시작지 목록
                        candidates.append((element[0].up_pos, (i, j)))
        if len(car.reserved) > 0:
            candidates += [(ele.up_pos, i) for i, ele in enumerate(car.reserved)] # 내가 예약한 승객의 시작지 목록
        if len(car.capacity) > 0: 
            candidates += [(ele.down_pos, i) for i, ele in enumerate(car.capacity)] # 내가 태운 승객의 도착지 목록
        if len(candidates) == 0: # 예외 처리, 후보자가 한명도 없는 경우 구역에 상관없이 출발지가 가장 가까운 사람
            for i, row in enumerate(customers):
                for j, element in enumerate(row):
                    candidates.append((element[0].up_pos, (i, j)))
        distances = [cal_distance(ele[0], now_pos) if cal_distance(ele[0], now_pos) != 0 else 1e8 for ele in candidates] # 각각 원소마다 현재 위치와의 거리 계산
        min_index = distances.index(min(distances))
        if isinstance(candidates[min_index][1], tuple): # 새로운 승객을 태우는 경우
            ct = candidates[min_index][1]
            des.append(candidates[min_index][0])
            car.reserved.append(customers[ct[0]][ct[1]][0]) # 튜플 위치의 승객
            car.passed_capacity = [ele + 1 for ele in car.passed_capacity]
            del customers[ct[0]][ct[1]]
            print(i,"th car", "go to ride new customer", car.reserved[-1].up_pos)
        else:
            des.append(candidates[min_index][0])
            print(i,"th car", "go to ride reserved customer/go to let down boarding customer")
    return des

In [253]:
car_number = 4

class Simulator(object):
    def __init__(self): 
        self.event_list = []
        self.car_number = car_number
        self.initial_pos = ['wykh9ugm', 'wy7uwygj', 'wy7vp8pe', 'wykj0ryg']
        self.car = [Car(index, [], [], self.initial_pos[index], [], []) for index in range(self.car_number)] # 차량 번호, 차량 용량, 차량 위치, 차량 목적지 리스트
        self.customer = [] # self.wt length, 맨 뒤에 최신 승객, [2분전, 1분전, 현재 승객], 여기는 geohash로
        self.customer_area = [[] for _ in range(len(area_inf))]
        self.time = 0
        self.trigger = {0: self.car_decision} # "demand" == 0
        for j in range(self.car_number):
            self.trigger[self.en(1, j)] = self.car_updown  # "stop_demand": 1
            self.trigger[self.en(2, j)] = self.car_moving  # "updown": 2
            self.trigger[self.en(3, j)] = self.car_moving  # "stop_middle": 3
        self.wt = 3 # 호출하고 예약이 안될때 최대로 기다리는 분
        self.n = 0 # 운행횟수 저장용
    
    def en(self, event_index, car_index): # event_number
        return (event_index - 1) * self.car_number + car_index + 1
    
    def ci(self, event_number):
        return (event_number - 1)% self.car_number
        
    def gen_customer(self):
        if len(self.customer) >= self.wt: self.customer.pop(0)
        self.customer.append([])
        for i, ele in enumerate(time_lambda[self.time//60]): # every_minuate
            temp_cus = Customer(i, self.time, ele[0], ele[1])
            for areai, areainf in enumerate(area_inf):
                startindex = cal_area_num(hashtoarrayindex(ele[0]))
                endindex = cal_area_num(hashtoarrayindex(ele[1]))
            if startindex != -1 and endindex != -1:
                self.customer[-1].append((temp_cus, startindex, endindex))
        print("custmoer generation", self.time, len(self.customer[-1]), self.customer[-1][0])
    
    def demand(self, max_time, dec_interval=60): # every_minuate
        for i in range(max_time):
            if i % dec_interval == 0: heappush(self.event_list, (i+0.1, 0)) # 결정은 가장 나중에
            if i % 60 == 0: heappush(self.event_list, (i-0.1, -1)) # 손님 반영은 가장 먼저
    
    def car_decision(self):
        paths = algorithm(self.customer, self.car)
        for i in range(car_number):
            store_des[i].append(paths[i])
        print("now_pos", self.time, [["now_pos", self.car[i].now_pos, "now_area", cal_area_num(hashtoarrayindex(self.car[i].now_pos))] for i in range(self.car_number)])
        for i in range(self.car_number):
            self.car[i].path = paths[i]
            store_capacity[i][self.time//60] = self.car[i].capacity
            print(i,"th car", "path", self.car[i].path)
        print("decision", self.time, "\n")
            
    def car_updown(self, index): 
        now_pos = self.car[index].now_pos
        cap = self.car[index].capacity
        pcap = self.car[index].passed_capacity
        reserved = self.car[index].reserved
        num = [0, 0] # 타거나 내리는 승객의 수
        cus_inf = [] # 승객 정보 외부 변수에 저장용
        
        remove_index = [] # 승객을 내리게 하는 부분
        for i, cus in enumerate(cap):
            if cus.down_pos == now_pos:
                print(index,"th car", "I let down him")
                cus_inf.append([cus, self.time])
                remove_index.append(i)
                num[1] += 1
                self.n += 1 # 운행이 완료된 횟수
        self.car[index].capacity = [cap[i] for i in range(len(cap)) if i not in remove_index] # 차량 용량 삭제
        self.car[index].passed_capacity = [pcap[i] for i in range(len(pcap)) if i not in remove_index] # 차량 용량 삭제
        
        remove_index = [] # 예약된 승객을 태우는 부분
        for i, cus in enumerate(reserved):
            if cus.up_pos == now_pos:
                print(index,"th car", "I ride him")
                cus_inf.append([cus, self.time])
                remove_index.append(i)
                self.car[index].capacity.append(cus)
                self.car[index].passed_capacity.append(0)
                num[0] += 1
        self.car[index].reserved = [reserved[i] for i in range(len(reserved)) if i not in remove_index]
        
        print(index,"th car", "updown", index, now_pos, "up_cus_num", num[0], "down_cus_num", num[1])
        if sum(num) > 0:
            for ele in cus_inf:
                if ele[0] not in store_cus[index]: store_cus[index][ele[0]] = [ele[1]]
                else: store_cus[index][ele[0]].append(ele[1])
            heappush(self.event_list, (self.time + 60, self.en(2, index))) # 승객을 태웠다면
        # 노쇼 등으로 태우지 못하더라도 예외 처리를 해야함, 에러 날까봐 미뤄둠
            
    def car_moving(self, index):
        car = self.car[index]
        print(index,"th car", "before moving", car.now_pos, cal_area_num(hashtoarrayindex(car.now_pos)))
        temp_path = cal_path(geohash.decode(car.now_pos), geohash.decode(car.path))
        store_path[index] += [[pos, False] for pos in temp_path[1:-1]]
        store_path[index].append([geohash.decode(car.path), True])
        left_time = cal_total_time(car.now_pos, car.path)
        car.now_pos = car.path
        print(index,"th car", "after moving", car.now_pos, cal_area_num(hashtoarrayindex(car.now_pos)), "left_time", left_time)
        heappush(self.event_list, (self.time + left_time, self.en(1, index))) 
            
    def run(self, max_time): # 모든 시간은 초 단위로 사용하자 나중에 확장
        for _ in range(10000):
            if len(self.event_list) == 0 or (max_time < int(abs(round(self.event_list[0][0], 0)))): break # 최대 시간 설정
            t, event = heappop(self.event_list)
            self.time = int(abs(round(t, 0))) 
            if event == -1: self.gen_customer()
            elif event == 0: self.trigger[event]()
            else: self.trigger[event](self.ci(event)) # car_index: (event - 1)% self.car_number 
                        
    def run_step(self, action, time): # 강화학습용
        self.action = action 
        self.run(time)
        cars_now_pos = [self.car[i].now_pos for i in range(self.car_number)]
        cars_now_cus = [self.car[i].capacity for i in range(self.car_number)]
        # return self.customer, cars_now_pos, cars_now_cus # 승객의 전체 위치, 현재 위치, 타고 있는 승객 정보
        
    def reset(self): # 승객이 발생하도록 이벤트를 넣어준다
        self.event_list = []
        self.car = [Car(index, [], [], self.initial_pos[index], [], []) for index in range(self.car_number)]
        self.customer = []
        self.demand(3600) # 1을 넣으면 1초 단위로 결정, 안 넣으면 1분 단위(승객이 생길 때)로 결정
        for index in range(self.car_number):
            heappush(self.event_list, (1, self.en(3, index))) # 초기 움직임 이니시에이팅
            store_path[index].append([geohash.decode(self.car[index].now_pos), False]) # 시작지점 지도에 찍기

In [254]:
global store_path
store_path = [[] for _ in range(car_number)]
global store_cus
store_cus = [{} for _ in range(car_number)]
global store_capacity
store_capacity = [{key: [] for key in range(60)} for _ in range(car_number)]
global store_des
store_des = [[] for _ in range(car_number)]

In [255]:
tra = Simulator()
tra.reset()

In [256]:
tra.run(3600)

custmoer generation 0 31 (<__main__.Customer object at 0x000001A02284D6D0>, 2, 3)
0 th car go to ride new customer wykhc04k
0 th car go to ride new customer wykhb4hq
0 th car go to ride new customer wy7vp8nq
0 th car go to ride new customer wykj0qnq
now_pos 0 [['now_pos', 'wykh9ugm', 'now_area', 0], ['now_pos', 'wy7uwygj', 'now_area', 1], ['now_pos', 'wy7vp8pe', 'now_area', 2], ['now_pos', 'wykj0ryg', 'now_area', 3]]
0 th car path wykhc04k
1 th car path wykhb4hq
2 th car path wy7vp8nq
3 th car path wykj0qnq
decision 0 

0 th car before moving wykh9ugm 0
cal_path 길찾기 성공
0 th car after moving wykhc04k 0 left_time 420
1 th car before moving wy7uwygj 1
cal_path 길찾기 성공
1 th car after moving wykhb4hq 1 left_time 540
2 th car before moving wy7vp8pe 2
cal_path 길찾기 성공
2 th car after moving wy7vp8nq 2 left_time 60
3 th car before moving wykj0ryg 3
cal_path 길찾기 성공
3 th car after moving wykj0qnq 3 left_time 60
custmoer generation 60 34 (<__main__.Customer object at 0x000001A0229ECF50>, 2, 3)
1 th 

In [257]:
import folium.plugins as plugins

def visual_ith_car_path(cari):
    last = 0
    for i, ele in enumerate(store_path[cari]):
        pos, bi = ele
        if bi == False:
            if i == 0:
                folium.Marker(pos, 
                  popup=f'CarNumber: {cari}', 
                  icon=folium.Icon(color='white', prefix='fa')
                 ).add_to(m)
        else:
            plugins.AntPath(
                locations=[ele[0] for ele in store_path[cari][last:i+1]], dash_array=[20, 30]
            ).add_to(m)
            last = i
    plugins.AntPath(
        locations=[ele[0] for ele in store_path[cari][last:]], dash_array=[20, 30]
    ).add_to(m)

    for i, temp_cus in enumerate(list(store_cus[cari].keys())):
        time_list = store_cus[cari][temp_cus]
        start, end = temp_cus.up_pos, temp_cus.down_pos
        folium.Marker(geohash.decode(start), 
          popup=f'WaitingTime: {(time_list[0]-temp_cus.time)//60}min', 
          icon=folium.Icon(icon = str(i+1), color='red', prefix='fa')
         ).add_to(m)
        if len(time_list) > 1:
            folium.Marker(geohash.decode(end), 
              popup=f'BoardingTime: {(time_list[1]-time_list[0])//60}min', 
              icon=folium.Icon(icon = str(i+1), color='blue', prefix='fa')
             ).add_to(m)

car_index = 1
m = folium.Map(
location=geohash.decode(tra.initial_pos[car_index]), 
zoom_start=12, tiles='cartodbpositron', width=800, height=500)
visual_ith_car_path(car_index) # 여기서 차량 번호를 바꾼다
m.fit_bounds(m.get_bounds())
m

In [239]:
# 시각화 리스트
# 동승으로 인한 도착지연시간(완)
# 구역 별 탑승 거절 비율(완)
# 평균 탑승/대기 시간(완)
# 평균 분별 탑승자 수(완)
# 총 탑승자 수/총 운행 수(완)
# 평균 동승자 기대값(완)
# 평균 정차 간 이동시간(완)
# 차량별 이동 거리(완)

In [238]:
# 동승으로 인한 탑승 지연 시간 및 시간 배율
# store_cus만 사용
boarding_time_compare = []
# store_cus
# 탑승한 승객의 승차 구역을 각각 카운트
for cus_per_car in store_cus:
    for tcus in cus_per_car:
        if len(cus_per_car[tcus]) == 2:
            a = cal_total_time(tcus.up_pos, tcus.down_pos)
            b = cus_per_car[tcus][1] - cus_per_car[tcus][0]
            boarding_time_compare.append((b, a))
print(boarding_time_compare)
average = []
diff = []
for ele in boarding_time_compare:
    average.append(ele[0]/ele[1])
    diff.append(ele[0] - ele[1])
print(average)
print(diff)
print("시간 지연 배율", int(sum(average)/len(average) * 100), "%")
print("평균 지연 시간", sum(diff)/len(diff)//60, "min")

[(2700, 1740), (180, 120), (120, 60), (180, 120), (720, 180), (480, 120), (600, 300), (180, 120), (1200, 420), (1260, 300), (1260, 300), (1380, 240), (1500, 120), (1500, 120), (600, 420), (1140, 180), (1200, 60), (960, 60), (600, 120), (1260, 480), (1980, 240), (2040, 300), (2220, 240), (2400, 240), (780, 1560), (540, 480)]
[1.5517241379310345, 1.5, 2.0, 1.5, 4.0, 4.0, 2.0, 1.5, 2.857142857142857, 4.2, 4.2, 5.75, 12.5, 12.5, 1.4285714285714286, 6.333333333333333, 20.0, 16.0, 5.0, 2.625, 8.25, 6.8, 9.25, 10.0, 0.5, 1.125]
[960, 60, 60, 60, 540, 360, 300, 60, 780, 960, 960, 1140, 1380, 1380, 180, 960, 1140, 900, 480, 780, 1740, 1740, 1980, 2160, -780, 60]
시간 지연 배율 566 %
평균 지연 시간 13.0 min


In [252]:
# 구역 별 탑승 거절 비율, 0프로로 만들려면 얼마나 차량이 있어야 할지 판단에 도움을 줄 수 있을 듯
# store_cus만 사용
total_boarded_cus_per_area = [0 for _ in range(len(area_inf))]
# store_cus
# 탑승한 승객의 승차 구역을 각각 카운트
for cus_per_car in store_cus:
    for tcus in cus_per_car:
        total_boarded_cus_per_area[cal_area_num(hashtoarrayindex(tcus.up_pos))] += 1
print("구역별 탑승자 수", total_boarded_cus_per_area)

total_up_cus_per_area = [0 for _ in range(len(area_inf))]
total_down_cus_per_area = [0 for _ in range(len(area_inf))]
for i in range(60):
    for tcus in time_lambda[i]:
        thash1 = tcus[0]
        thash2 = tcus[1]
        total_up_cus_per_area[cal_area_num(hashtoarrayindex(thash1))] += 1
        total_down_cus_per_area[cal_area_num(hashtoarrayindex(thash2))] += 1
print("구역별 탑승 수요자 수", total_up_cus_per_area)
print("구역별 하차 수요자 수", total_down_cus_per_area)
rejection_pro = [100-total_boarded_cus_per_area[i]/total*100 for i, total in enumerate(total_cus_per_area)]
print("거절 비율", rejection_pro)
recommended_car_number = [int(100/(100 - i)) for i in rejection_pro]
print("추천하는 차량 대수", recommended_car_number)
# 각 구역에서 호출한 승객 수

구역별 탑승자 수 [9, 11, 17, 13]
구역별 탑승 수요자 수 [243, 413, 732, 455]
구역별 하차 수요자 수 [197, 357, 0, 1289]
거절 비율 [96.29629629629629, 97.3365617433414, 97.6775956284153, 97.14285714285714]
추천하는 차량 대수 [26, 37, 43, 34]


In [242]:
# 평균 대기 시간, 평균 탑승 시간
waiting_time_dis = []
boarding_time_dis = []
for cus_per_car in store_cus:
    for tcus in cus_per_car:
        ele = cus_per_car[tcus]
        waiting_time_dis.append(ele[0] - 1 - tcus.time)
        if len(ele) > 1:
            boarding_time_dis.append(ele[1] - ele[0])
print(waiting_time_dis)
print(boarding_time_dis)
print("mean_waiting_time", sum(waiting_time_dis)/len(waiting_time_dis)//60, "min")
print("mean_boarding_time", sum(boarding_time_dis)/len(boarding_time_dis)//60, "min")

[420, 120, 540, 120, 180, 60, 180, 180, 60, 540, 180, 180, 660, 240, 240, 60, 60, 120, 1260, 180, 60, 120, 180, 180, 120, 60, 120, 60, 180, 120, 180, 1980, 180, 180, 240, 120, 180, 60, 60, 120, 120, 480, 180, 60, 360, 60, 120, 120, 240, 240]
[2700, 180, 120, 180, 720, 480, 600, 180, 1200, 1260, 1260, 1380, 1500, 1500, 600, 1140, 1200, 960, 600, 1260, 1980, 2040, 2220, 2400, 780, 540]
mean_waiting_time 4.0 min
mean_boarding_time 18.0 min


In [243]:
# 평균 분별 탑승자수
cus_per_min = [[0 for _ in range(60)] for _ in range(car_number)]
for j, cus_per_car in enumerate(store_capacity):
    for i in range(60):
        cus_per_min[j][i] += len(cus_per_car[i])
print(cus_per_min)
mean_capacity = []
for i in range(car_number):
    mean_capacity.append(int(sum(cus_per_min[i])/60))
print("평균 분별 탑승자 수", mean_capacity)

[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 4, 4, 3, 3, 2, 2, 3, 3, 3, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4], [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 6, 6, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 6, 6, 6, 5, 5, 4, 4, 4, 4, 3, 3, 2, 2, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9, 9, 7, 7, 6], [0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 5, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6]]
평균 분별 탑승자 수 [6, 2, 4, 4]


In [244]:
# 총 탑승자 수, 운행 완료 수
total_boarding_cus = []
total_complete_cus = [0 for _ in range(car_number)]
for i, cus_per_car in enumerate(store_cus):
    total_boarding_cus.append(len(cus_per_car))
    for cus in cus_per_car:
        ele = cus_per_car[cus]
        if len(ele) > 1:
            total_complete_cus[i] += 1
print("차량별 탑승자 수", total_boarding_cus)
print("차량별 운행완료 수", total_complete_cus)

차량별 탑승자 수 [9, 11, 17, 13]
차량별 운행완료 수 [1, 7, 11, 7]


In [245]:
already_cus_num = [[] for _ in range(car_number)]
for i, cus_per_car in enumerate(store_cus):
    for cus in cus_per_car:
        ele = cus_per_car[cus]
        minu = (ele[0]-1)//60
        already_cus_num[i].append(len(store_capacity[i][minu])) 
print(already_cus_num)
already_cus_num = [int(sum(percar)/len(percar)) for percar in already_cus_num]
print("차량별 탔을때 평균 동승자 수", already_cus_num)

[[0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 0, 0, 0, 1, 2, 3, 4, 2, 2, 2], [0, 1, 2, 3, 4, 4, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 4, 2, 2, 3, 4, 5]]
차량별 탔을때 평균 동승자 수 [4, 1, 3, 3]


In [246]:
between_time = [[] for _ in range(car_number)]
moving_dis = [0 for _ in range(car_number)]
for i in range(car_number):
    for j in range(len(store_des[i]) - 1):
        between_time[i].append(cal_total_time(store_des[i][j], store_des[i][j+1]))
        moving_dis[i] += cal_distance(store_des[i][j], store_des[i][j+1])
average_bt = [int(sum(ele)/len(ele)) for ele in between_time]
print(between_time)
print("차량별 다음 정차까지 평균 시간", average_bt, "seconds")
moving_dis = [ele//1000 for ele in moving_dis]
print("차량별 이동 거리", moving_dis, "km")

[[60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 360, 300, 60, 60, 60, 60, 300, 360, 1680, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 1560, 60, 60, 60, 60, 60, 60], [60, 60, 300, 120, 60, 60, 60, 60, 60, 540, 60, 60, 60, 60, 120, 60, 60, 180, 120, 120, 60, 60, 60, 60, 60, 180, 60, 600, 60, 300, 120, 120, 360, 300, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 240, 60, 60, 60, 60, 180, 780, 60, 60, 60, 60, 60, 60, 60], [60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 180, 420, 60, 60, 60, 60, 60, 60, 60, 60, 120, 60, 60, 60, 180, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60], [60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 480, 60, 60, 60, 60, 60, 60, 60, 60, 60, 240, 60, 60, 300, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 60, 60, 240, 60, 60, 60, 60, 60, 120, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 120, 180, 60]]
차량별 다음 정차